In [1]:
%load_ext autotime

time: 0 ns (started: 2023-06-05 12:24:06 +07:00)


In [2]:
import numpy as np
from fairscale.experimental.tooling.layer_memory_tracker import LayerwiseMemoryTracker
from fastai.vision.all import *

time: 18.4 s (started: 2023-06-05 12:24:06 +07:00)


In [3]:
import pathlib
temp = pathlib.PosixPath
pathlib.PosixPath = pathlib.WindowsPath

time: 0 ns (started: 2023-06-05 12:24:25 +07:00)


In [15]:
# Set the size of the image
img_size = (64, 64)

def create_random_data():
    # Generate a random NumPy array
    rand_arr = np.random.rand(*img_size, 3) * 255
    rand_arr = rand_arr.astype(np.uint8)
    return rand_arr

def get_model_file_size(model): 
    param_size = 0
    for param in model.parameters():
        param_size += param.nelement() * param.element_size()

    model_size = param_size
    print('Model size: {} Byte'.format(model_size))

# Create your learner and load your model
model = load_learner("../../../../model/nvidia-t4/VGG16_bn_accum4.pkl")

get_model_file_size(model.model)

# Set the model to evaluation mode
model.eval()


# Get activation memory
tracker = LayerwiseMemoryTracker()
tracker.monitor(model)

x = create_random_data()
preds = model.predict(x)


tracker.stop()


Model size: 537239848 Byte


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

time: 1.81 s (started: 2023-06-05 12:36:57 +07:00)


In [ ]:
forward_act = sum(t.event.memory_activations for t in tracker.forward_traces) 
print(f"Activation Memory: {forward_act} Byte")

Activation Memory: 9480232 Byte
time: 0 ns (started: 2023-05-29 16:20:23 +07:00)
